In [24]:
#importando librerias que se piensan utilizar
import numpy as np
import pandas as pd
import seaborn as sns
import re


In [77]:
#Visualizar los primero prrimeros registros
df_properatti = pd.read_csv('properatti.csv')
df_properatti.head(4)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...


In [79]:
#renombrado primer columana que no tiene nombre
df_properatti.rename(columns={'Unnamed: 0':'id'}, inplace=True)


In [81]:
#Me quedo con la columna description y titulo
dfDescripcion = pd.DataFrame(df_properatti['description'].str.lower())
dfTitle = pd.DataFrame(df_properatti['title'].str.lower())
df_id = pd.DataFrame(df_properatti['id'])

# Creacion de diccionario de palabras, pasa luego reemplazarlas

In [82]:
#Creamos un diccionario 
dic = {
  'uno': '1',
  'dos': '2',
  'tres': '3',
  'cuatro': '4',
  'cinco': '5',
  'seis': '6',
  'siete': '7',
  'ocho': '8',
  'nueve': '9',
  'diez': '10',
  '1er': "1",
  '2do': "2",
  '3er': "3",
  '4to': "4",
  'º':"",
  'exp=': "expensas ",
  'abl=': " abl",
  'monoh':' monoambiente ',
  'monoambiente':'1 ambiente',
  'depto':' departamento ',
  'dorm':' dormitorios ',
  ' amb ':' ambientes ',
   'amb.':' ambientes ',
  'ambiente':' ambientes ',
  'ambientess':' ambientes ',
  'dormitorioss':' dormitorios ',
  'dormitorio':' dormitorios ',
  'habitacion':' dormitorios ',
  'habitaciones':' dormitorios ',
  'baa  o':' banos ',
  'usd':' u$d ',
  'u$s':' u$d ',
  'garage':' cochera ',
  'garaje':' cochera ', 
  'laundry':' lavadero ',
  'piscina':' pileta ',
  'swit':' suite ',
  'dormitorios suite':' dormitorios en suite ',
  'pre-venta':' emprendimiento ',
  'desde el pozo':' emprendimiento',
  'venta en pozo':' emprendimiento',
  'construccin':' construccion',
  'proyecto en construccion':'emprendimiento',
  'financiado':'emprendimiento',
  'cuota':'emprendimiento',
  'exp': ' expensas',
  'expensa':  'expensas',
  'posibilidad de cochera':  '',
  'baul': 'baulera'
}



In [83]:
#aplicamos diccionario sobre la columna descripcion
for key, valor in dic.items():
    dfDescripcion.replace(key, valor, regex=True, inplace=True)

In [84]:
#aplicamos diccionario sobre la columna descripcion
for key, valor in dic.items():
    dfTitle.replace(key, valor, regex=True, inplace=True)

In [87]:
dfResult = dfDescripcion.join(dfTitle)
dfResult = dfResult.join(df_id)


# Generacion de Columnas Dummy

In [88]:
#Agrego columna dummy si tiene pileta
patron = r'(pileta|pisci)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["pileta"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [89]:
#Agrego columna dummy si amenities
patron = r'(ameni|gimn|gym|lavadero|sum|usos mult|solarium)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["amenities"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [90]:
#Agrego columna dummy si tiene garage
patron = r'(garage|cocher|cochera)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["cochera"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [91]:
#Agrego columna dummy para saber si tiene garage
patron = r'(pozo|cuota|proyec|emprendim)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["pozo_en_descrip"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [92]:
#Agrego columna dummy para saber si tiene garage
patron = r'(pozo|cuota|proyec|emprendim)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["pozo_en_title"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [93]:
#Agrego columna dummy para saber si es a estrenar
patron = r'(a est|estren|a estrenar|estrenar)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["a_estrenar"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [94]:
#Agrego columna dummy para saber si es a estrenar
patron = r'(expensas|expensas)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["expensas"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [95]:
#Agrego columna dummy para saber si tiene baulera
patron = r'(baulera|baul)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["baulera"] = 1 * dfResult.description.apply(lambda x: bool(regex.search(str(x))))

In [96]:
#Agrego columna dummy para saber si es apto credito
patron = r'(apto credito|aptocredito)'
regex = re.compile(patron, flags=re.IGNORECASE)
dfResult["apto_credito"] = 1 * dfResult.title.apply(lambda x: bool(regex.search(str(x)))) 

In [70]:
dfResult

,description,title,pileta,amenities,cochera,pozo_en_descrip,pozo_en_title,a_estrenar,expensas,baulera,apto_credito
0,2 ambientes s entes tipo casa planta baja po...,2 ambientes s entes s entes tipo casa sin ...,0,1,0,0,0,0,0,0,0
1,venta de departamento en décimo piso al frente...,venta departamento 2 dormitorios s itori...,0,1,0,0,0,1,0,0,0
2,2 ambientes s entes 3 piso lateral living co...,2 ambientes s entes s entes 3 piso con asc...,0,1,0,0,0,0,1,0,1
3,ph 3 ambientes s entes con patio. hay 3 dep...,ph 3 ambientes s entes s cfte. reciclado,0,0,0,0,0,0,0,0,0
4,departamento con fantástica iluminación natura...,departamento 2 ambientes s entes s entes...,0,0,0,0,0,0,0,0,0
5,"casa en el perímetro del barrio 338, ubicada e...","casa barrio 338. sobre calle 3 de caballería, ...",0,1,0,0,0,0,0,1,0
6,muy buen ph al frente con entrada independient...,muy buen ph al frente 2 dormitorios s itor...,0,1,0,0,0,1,1,0,0
7,excelente 1 ambientes s ente a estrenar ampl...,jose hernandez 1400 1 ambientes s entes s ...,1,1,0,0,0,1,0,0,0
8,excelente 2 ambientes s entes estrenar ampl...,jose hernandez 1400 2 ambientes s entes ...,1,1,0,0,0,1,0,0,0
9,mednoza al 7600a una cuadra de calle mendozawh...,white 7637 - 2 dormitorios s itorios s ito...,0,1,0,0,0,0,0,0,0


In [97]:
dflimpio = pd.read_csv('propiedades_v1.csv')

In [98]:
dfFinal = pd.merge(dflimpio, dfResult, on='id', how='left')

In [100]:
dfFinal.drop(["description","title"], axis=1)

,id,Transaccion,Tipo Propiedad,Piso,Ambientes,Expensas,Pais,Localidad/Provincia(Arg),Partido,Zona(CABA),...,Requiere Analisis,pileta,amenities,cochera,pozo_en_descrip,pozo_en_title,a_estrenar,expensas,baulera,apto_credito
0,0,venta,ph,NaN,NaN,NaN,argentina,capital federal,NaN,NaN,...,NaN,0,1,0,0,0,0,0,0,0
1,1,venta,departamento,NaN,NaN,NaN,argentina,bs.as. g.b.a. zona sur,la plata,NaN,...,Sup Total,0,1,0,0,0,1,0,0,0
2,2,venta,departamento,NaN,NaN,NaN,argentina,capital federal,NaN,NaN,...,NaN,0,1,0,0,0,0,1,0,1
3,3,venta,ph,NaN,NaN,NaN,argentina,capital federal,NaN,NaN,...,Sup Total,0,0,0,0,0,0,0,0,0
4,4,venta,departamento,NaN,NaN,NaN,argentina,buenos aires costa atlántica,mar del plata,NaN,...,NaN,0,0,0,0,0,0,0,0,0
5,5,venta,casa,NaN,NaN,NaN,argentina,entre ríos,gualeguaychú,NaN,...,NaN,0,1,0,0,0,0,0,1,0
6,6,venta,ph,NaN,NaN,NaN,argentina,bs.as. g.b.a. zona norte,vicente lópez,NaN,...,NaN,0,1,0,0,0,1,1,0,0
7,7,venta,departamento,NaN,NaN,NaN,argentina,capital federal,NaN,NaN,...,NaN,1,1,0,0,0,1,0,0,0
8,8,venta,departamento,NaN,NaN,NaN,argentina,capital federal,NaN,NaN,...,NaN,1,1,0,0,0,1,0,0,0
9,9,venta,casa,NaN,NaN,NaN,argentina,santa fe,rosario,NaN,...,Sup Total,0,1,0,0,0,0,0,0,0


In [102]:
dfFinal.to_dense().to_csv("propiedades_v2.csv", index = False, sep=',', encoding='utf-8')